In [5]:
import talib as ta
import pandas as pd
import numpy as np

In [6]:
# read the data from the CSV files
train_data = pd.read_csv('train_data_50.csv') 

# change date into datetime objects
train_data['Date'] = pd.to_datetime(train_data['Date'])

# set indexes 
train_data.set_index(["Ticker", "Date"], inplace=True)

# sort data by tickers
tickers = sorted(train_data.index.get_level_values('Ticker').unique())

In [7]:
#create array of close prices 
tickers = sorted(train_data.index.get_level_values('Ticker').unique())

open_prices = []

for ticker in tickers:
    stock_close_data = train_data.loc[ticker]["Open"]
    open_prices.append(stock_close_data.values)

open_prices = np.stack(open_prices)
print(open_prices.shape)
print(open_prices)

# array for final return - trades
trades = np.zeros_like(open_prices)
trades


(50, 1452)
[[17.04294995 17.25096871 16.96448663 ... 67.35797313 68.06231506
  68.62432851]
 [12.20377331 12.24209241 12.04319784 ... 27.93657064 28.10079496
  28.43107009]
 [27.76139563 28.32888582 28.12086706 ... 45.28424827 45.45394932
  45.20031238]
 ...
 [12.73111928 12.80593376 12.77856257 ... 40.5016405  40.69141088
  40.9340999 ]
 [ 8.02149948  8.0908393   8.10908629 ... 45.21855798 46.12362377
  46.14186937]
 [20.12126341 20.38584965 20.38584965 ... 57.08475709 57.0592138
  57.54641278]]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:

for stock in range(len(open_prices)):
    #we might have to use open data as the first paramter

    macd, signal_line, _ = ta.MACD(open_prices[stock], fastperiod=12, slowperiod=26, signalperiod=9)

    #if macd crossed above signal line then buy, vice versa sell
    for day in range(1, len(open_prices[0]) - 1):
        if macd[day] > signal_line[day] and macd[day - 1] <= signal_line[day - 1]:
            trades[stock][day + 1] = 5  # Buy signal
        elif macd[day] < signal_line[day] and macd[day - 1] >= signal_line[day - 1]:
            trades[stock][day + 1] = -5  # Sell signal
        else:
            trades[stock][day+1] = 0

In [14]:
from eval_algo import eval_actions

portfolio_value, sharpe_ratio = eval_actions(trades, open_prices, cash=25000, verbose=True)
print(f"\nPortfolio value: {portfolio_value}")
print(f"Sharpe ratio: {sharpe_ratio}")

final portfolio value: 25406.70327760566
cash: 20676.863216470836
positions: [0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, -270.0, -300.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, -285.0, 5.0, 5.0, -290.0, -250.0, 5.0, -320.0, -310.0, 5.0, -315.0, -280.0, 5.0, -265.0, -235.0, -255.0, 5.0, 0.0, 5.0, 5.0, 5.0, -280.0, -255.0, 0.0, -260.0, -275.0, 5.0, 5.0, -245.0]
short position info: defaultdict(<class 'list'>, {7: [[13.147436628357063, 5.0]], 8: [], 22: [], 26: [[10.731219415385269, 5.0]], 32: [], 36: [], 42: [[10.30240816561338, 5.0]], 45: [], 25: [], 28: [], 29: [], 31: [[66.45473106319916, 5.0]], 34: [], 35: [], 43: [], 46: [], 49: []})
short value: 1.19519392166465

Portfolio value: [25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 24975.205392201104, 24992.45729434892